<a href="https://colab.research.google.com/github/2303A52046/GenAI_A1_2303A52046/blob/main/GENERATIVE_AI_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
# Clone InstantID repo
!git clone https://github.com/InstantID/InstantID.git
%cd InstantID

# Install requirements
!pip install -r requirements.txt
!pip install insightface onnxruntime

# Create necessary folders
!mkdir -p checkpoints/InstantID checkpoints/stable-diffusion-v1-5

# Download InstantID face encoder
!wget -O checkpoints/InstantID/buffalo_l.zip https://huggingface.co/InstantX/InstantID/resolve/main/insightface/models/buffalo_l.zip
!unzip checkpoints/InstantID/buffalo_l.zip -d checkpoints/InstantID/

# Download IP-Adapter model
!wget -O checkpoints/ip-adapter.bin https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter.bin

# Install Git LFS to download Stable Diffusion 1.5
!apt-get install git-lfs
!git lfs install


Cloning into 'InstantID'...
remote: Enumerating objects: 540, done.
remote: Counting objects: 100% (329/329), done.
remote: Compressing objects: 100% (168/168), done.
remote: Total 540 (delta 253), reused 161 (delta 161), pack-reused 211 (from 1)
Receiving objects: 100% (540/540), 154.98 MiB | 37.97 MiB/s, done.
Resolving deltas: 100% (297/297), done.
/content/InstantID/InstantID
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
--2025-04-07 15:55:40--  https://huggingface.co/InstantX/InstantID/resolve/main/insightface/models/buffalo_l.zip
Resolving huggingface.co (huggingface.co)... 3.168.73.106, 3.168.73.111, 3.168.73.129, ...
Connecting to huggingface.co (huggingface.co)|3.168.73.106|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-04-07 15:55:40 ERROR 404: Not Found.

Archive:  checkpoints/InstantID/buffalo_l.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it con

In [34]:
from huggingface_hub import login
login()


In [35]:
!git clone https://huggingface.co/runwayml/stable-diffusion-v1-5 checkpoints/stable-diffusion-v1-5


Cloning into 'checkpoints/stable-diffusion-v1-5'...
remote: Repository not found
fatal: repository 'https://huggingface.co/runwayml/stable-diffusion-v1-5/' not found


In [39]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pipe.to(device)


Pipelines loaded with `dtype=torch.float16` cannot run with `cpu` device. It is not recommended to move them to `cpu` as running them will fail. Please make sure to use an accelerator to run the pipeline in inference, due to the lack of support for`float16` operations on this device in PyTorch. Please, remove the `torch_dtype=torch.float16` argument, or use another device for inference.
Pipelines loaded with `dtype=torch.float16` cannot run with `cpu` device. It is not recommended to move them to `cpu` as running them will fail. Please make sure to use an accelerator to run the pipeline in inference, due to the lack of support for`float16` operations on this device in PyTorch. Please, remove the `torch_dtype=torch.float16` argument, or use another device for inference.
Pipelines loaded with `dtype=torch.float16` cannot run with `cpu` device. It is not recommended to move them to `cpu` as running them will fail. Please make sure to use an accelerator to run the pipeline in inference, du

StableDiffusionXLInstantIDImg2ImgPipeline {
  "_class_name": "StableDiffusionXLInstantIDImg2ImgPipeline",
  "_diffusers_version": "0.32.2",
  "_name_or_path": "stabilityai/stable-diffusion-xl-base-1.0",
  "controlnet": [
    "diffusers",
    "ControlNetModel"
  ],
  "feature_extractor": [
    null,
    null
  ],
  "force_zeros_for_empty_prompt": true,
  "image_encoder": [
    null,
    null
  ],
  "requires_aesthetics_score": false,
  "scheduler": [
    "diffusers",
    "EulerDiscreteScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "text_encoder_2": [
    "transformers",
    "CLIPTextModelWithProjection"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "tokenizer_2": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [ ]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


Looking in indexes: https://download.pytorch.org/whl/cu121
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
     ━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/780.5 MB 119.6 kB/s eta 1:34:07

In [4]:
import torch  # ✅ Add this line
from diffusers.models import ControlNetModel

controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/sd-controlnet-openpose",
    torch_dtype=torch.float16
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
from insightface.app import FaceAnalysis
from diffusers.models import ControlNetModel
from pipeline_stable_diffusion_xl_instantid_img2img import StableDiffusionXLInstantIDImg2ImgPipeline

# Initialize face encoder
app = FaceAnalysis(name='buffalo_l', root='checkpoints/InstantID', providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
app.prepare(ctx_id=0, det_size=(640, 640))

# Load ControlNet
controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/sd-controlnet-openpose",  # or your own model
    torch_dtype=torch.float16
)

# Load base SDXL pipeline
pipe = StableDiffusionXLInstantIDImg2ImgPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    controlnet=controlnet,
    torch_dtype=torch.float16
)
pipe.cuda()
pipe.load_ip_adapter_instantid('./checkpoints/ip-adapter.bin')


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: checkpoints/InstantID/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: checkpoints/InstantID/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: checkpoints/InstantID/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: checkpoints/InstantID/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: checkpoints/InstantID/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 64

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [2]:
!wget https://github.com/InstantID/InstantID/raw/main/pipeline_stable_diffusion_xl_instantid_img2img.py

--2025-04-07 16:16:47--  https://github.com/InstantID/InstantID/raw/main/pipeline_stable_diffusion_xl_instantid_img2img.py
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/instantX-research/InstantID/raw/main/pipeline_stable_diffusion_xl_instantid_img2img.py [following]
--2025-04-07 16:16:47--  https://github.com/instantX-research/InstantID/raw/main/pipeline_stable_diffusion_xl_instantid_img2img.py
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/instantX-research/InstantID/main/pipeline_stable_diffusion_xl_instantid_img2img.py [following]
--2025-04-07 16:16:47--  https://raw.githubusercontent.com/instantX-research/InstantID/main/pipeline_stable_diffusion_xl_instantid_img2img.py
Resolving raw.githubusercontent.com (raw.githubusercontent